# Mapping des Équipes - World Cup ETL

**Auteur** : Romain  
**Date** : 16/12/2025

## Objectif
Créer le référentiel des équipes nationales avec leurs aliases pour harmoniser les noms à travers tous les datasets.

## 1. Imports et Configuration

In [1]:
import pandas as pd
import json
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import os

# Chemins
DATA_PATH = Path('../data/raw/')

In [2]:
# Connexion base de données
load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME')

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

def get_engine():
    return create_engine(DATABASE_URL)

# Test connexion
try:
    engine = get_engine()
    with engine.connect() as conn:
        print("Connecté à la base de données")
except Exception as e:
    print(f"Erreur: {e}")

Connecté à la base de données


## 2. Extraction des équipes de chaque dataset

In [3]:
# Dataset 1930-2010
df_1930_2010 = pd.read_csv(DATA_PATH / 'matches_19302010 (1).csv')
equipes_1930_2010 = set(df_1930_2010['team1'].unique()) | set(df_1930_2010['team2'].unique())
print(f"Équipes 1930-2010 : {len(equipes_1930_2010)}")

Équipes 1930-2010 : 262


In [4]:
# Dataset 2014
df_2014 = pd.read_csv(DATA_PATH / 'WorldCupMatches2014 (1).csv', sep=';', encoding='utf-8')
equipes_2014 = set(df_2014['Home Team Name'].unique()) | set(df_2014['Away Team Name'].unique())
print(f"Équipes 2014 : {len(equipes_2014)}")

Équipes 2014 : 32


In [5]:
# Dataset 2018
with open(DATA_PATH / 'data_2018.json', 'r', encoding='utf-8') as f:
    data_2018 = json.load(f)

equipes_2018 = {team['name'] for team in data_2018['teams']}
print(f"Équipes 2018 : {len(equipes_2018)}")

Équipes 2018 : 32


In [6]:
# Toutes les équipes uniques (brutes)
toutes_equipes_brutes = equipes_1930_2010 | equipes_2014 | equipes_2018
print(f"\nTotal équipes brutes : {len(toutes_equipes_brutes)}")
print("\nListe triée :")
for eq in sorted(toutes_equipes_brutes):
    print(f"  - '{eq}'")



Total équipes brutes : 294

Liste triée :
  - '1'
  - '2'
  - '3'
  - '4'
  - '5'
  - '6'
  - '7'
  - '8'
  - 'A'
  - 'A1'
  - 'A2'
  - 'Afghanistan (افغانستان)'
  - 'Albania (Shqipëri)'
  - 'Algeria'
  - 'Algeria (الجزائر)'
  - 'American Samoa'
  - 'Andorra'
  - 'Angola'
  - 'Anguilla'
  - 'Antigua'
  - 'Antigua and Barbuda'
  - 'Argentina'
  - 'Armenia (Հայաստան)'
  - 'Aruba'
  - 'Australia'
  - 'Austria (Österreich)'
  - 'Azerbaijan (Azərbaycan)'
  - 'B'
  - 'B1'
  - 'B2'
  - 'Bahamas'
  - 'Bahrain (البحرين)'
  - 'Bangladesh (বাংলাদেশ)'
  - 'Barbados'
  - 'Belarus (Беларусь)'
  - 'Belgium'
  - 'Belgium (België)'
  - 'Belize'
  - 'Benin (Bénin)'
  - 'Bermuda'
  - 'Bolivia'
  - 'Bosnia-Herzegovina (Bosna i Hercegovina)'
  - 'Botswana'
  - 'Brazil'
  - 'Brazil (Brasil)'
  - 'British Virgin Islands'
  - 'Brunei (بروني)'
  - 'Bulgaria (България)'
  - 'Burkina Faso'
  - 'Burundi'
  - 'C'
  - 'C1'
  - 'C2'
  - 'Cambodia (កម្ពុជា)'
  - 'Cameroon'
  - 'Cameroon (Cameroun)'
  - 'Canada'
  - 

## 3. Chargement des confédérations depuis GITHUB

In [7]:
# =============================================================================
# TÉLÉCHARGEMENT DES CONFÉDÉRATIONS DEPUIS GITHUB
# Source : https://github.com/cnc8/fifa-world-ranking
# Ce fichier est généré automatiquement depuis fifa.com
# =============================================================================

import requests
from datetime import datetime

print("=" * 60)
print("TÉLÉCHARGEMENT DES CONFÉDÉRATIONS FIFA (source: GitHub)")
print("=" * 60)

# URL du CSV brut sur GitHub (scraper FIFA officiel)
# Ce fichier est généré par un scraper de fifa.com
github_csv_url = "https://raw.githubusercontent.com/cnc8/fifa-world-ranking/master/fifa_ranking-2020-12-10.csv"

print(f"Source : {github_csv_url}")
print(f"Date de téléchargement : {datetime.now().strftime('%Y-%m-%d %H:%M')}")

try:
    # Télécharger le CSV
    response = requests.get(github_csv_url)
    response.raise_for_status()
    
    # Sauvegarder localement pour traçabilité
    local_path = Path('../data/reference/fifa_ranking_source.csv')
    local_path.parent.mkdir(parents=True, exist_ok=True)
    local_path.write_bytes(response.content)
    print(f"\nFichier sauvegardé : {local_path}")
    
    # Charger avec pandas
    df_fifa_github = pd.read_csv(local_path)
    print(f"Dataset chargé : {len(df_fifa_github)} lignes")
    print(f"Colonnes : {df_fifa_github.columns.tolist()}")
    
except Exception as e:
    print(f"Erreur : {e}")
    print("\nEssayons une autre méthode...")

TÉLÉCHARGEMENT DES CONFÉDÉRATIONS FIFA (source: GitHub)
Source : https://raw.githubusercontent.com/cnc8/fifa-world-ranking/master/fifa_ranking-2020-12-10.csv
Date de téléchargement : 2025-12-17 15:58

Fichier sauvegardé : ..\data\reference\fifa_ranking_source.csv
Dataset chargé : 62424 lignes
Colonnes : ['id', 'rank', 'country_full', 'country_abrv', 'total_points', 'previous_points', 'rank_change', 'confederation', 'rank_date']


In [8]:
# =============================================================================
# EXTRACTION DU MAPPING PAYS -> CONFÉDÉRATION
# Depuis le dataset GitHub (source: fifa.com via scraper)
# =============================================================================

# Vérifier que la colonne confederation existe
if 'confederation' in df_fifa_github.columns:
    print("Colonne 'confederation' trouvée !")
    
    # Prendre la dernière date disponible
    latest_date = df_fifa_github['rank_date'].max()
    df_latest = df_fifa_github[df_fifa_github['rank_date'] == latest_date].copy()
    
    print(f"Date du classement : {latest_date}")
    print(f"Nombre de pays : {len(df_latest)}")
    
    # Créer le mapping pays -> confédération
    CONFEDERATIONS_FIFA = df_latest.set_index('country_full')['confederation'].to_dict()
    
    # Afficher la répartition
    print("\n" + "=" * 60)
    print("RÉPARTITION PAR CONFÉDÉRATION (source: fifa.com)")
    print("=" * 60)
    print(df_latest['confederation'].value_counts().to_string())
    
    # Exemples par confédération
    print("\n" + "-" * 60)
    print("EXEMPLES PAR CONFÉDÉRATION")
    print("-" * 60)
    for conf in df_latest['confederation'].unique():
        countries = df_latest[df_latest['confederation'] == conf]['country_full'].head(5).tolist()
        print(f"{conf}: {', '.join(countries)}")
else:
    print("Colonne 'confederation' non trouvée !")
    print(f"Colonnes disponibles : {df_fifa_github.columns.tolist()}")

Colonne 'confederation' trouvée !
Date du classement : 2020-12-10
Nombre de pays : 210

RÉPARTITION PAR CONFÉDÉRATION (source: fifa.com)
confederation
UEFA        55
CAF         54
AFC         46
CONCACAF    35
OFC         10
CONMEBOL    10

------------------------------------------------------------
EXEMPLES PAR CONFÉDÉRATION
------------------------------------------------------------
CAF: Equatorial Guinea, Tanzania, Burundi, Lesotho, Botswana
UEFA: Latvia, Andorra, Faroe Islands, Azerbaijan, Estonia
AFC: Myanmar, Chinese Taipei, Maldives, Yemen, Kuwait
CONCACAF: St. Kitts and Nevis, Suriname, Nicaragua, Guatemala, Antigua and Barbuda
OFC: Solomon Islands, New Zealand, New Caledonia, American Samoa, Samoa
CONMEBOL: Venezuela, Paraguay, Peru, Brazil, Argentina


## 5. Équipes historiques (Complément manuel)

Ces équipes n'existent plus dans le classement FIFA actuel mais ont participé à des Coupes du Monde.

**Source** : [FIFA World Cup Archives](https://www.fifa.com/tournaments/mens/worldcup)

**Règle appliquée** : Conformément aux règles FIFA/UEFA, chaque équipe historique reste une entité distincte pour l'attribution des matchs. Le successeur FIFA est documenté à titre informatif pour les statistiques consolidées.

In [9]:
# =============================================================================
# ÉQUIPES HISTORIQUES ET LEURS SUCCESSEURS FIFA
# Ces équipes n'existent plus dans le classement FIFA actuel
# Format: "Équipe dissoute": (année_dissolution, "successeur_FIFA", "confédération")
# =============================================================================

HISTORICAL_TEAMS = {
    # Europe (UEFA)
    "Soviet Union": (1991, "Russia", "UEFA"),
    "Yugoslavia": (2003, "Serbia", "UEFA"),
    "Czechoslovakia": (1993, "Czech Republic", "UEFA"),
    "Serbia-Montenegro": (2006, "Serbia", "UEFA"),
    "GDR": (1990, "Germany", "UEFA"),  # Allemagne de l'Est
    "FRG": (1990, "Germany", "UEFA"),  # Allemagne de l'Ouest (= BRD)
    "Saarland": (1956, "Germany", "UEFA"),  # Protectorat de la Sarre
    "Irish Free State": (1936, "Republic of Ireland", "UEFA"),  # Nom FIFA: Republic of Ireland
    
    # Afrique (CAF)
    "Zaire": (1997, "Congo DR", "CAF"),  # Nom FIFA: Congo DR
    "Dahomey": (1975, "Benin", "CAF"),
    "Upper Volta": (1984, "Burkina Faso", "CAF"),
    "Rhodesia": (1980, "Zimbabwe", "CAF"),
    
    # Asie (AFC)
    "Dutch East Indies": (1945, "Indonesia", "AFC"),
    "North Yemen": (1990, "Yemen", "AFC"),
    "South Yemen": (1990, "Yemen", "AFC"),
    "South Vietnam": (1976, "Vietnam", "AFC"),
    
    # Amérique (CONCACAF / CONMEBOL)
    "Dutch Guyana": (1975, "Suriname", "CONMEBOL"),  # Note: Suriname maintenant CONCACAF
    "Dutch Antilles": (2010, None, "CONCACAF"),  # Dissous, pas de successeur unique
    
    # Océanie (OFC)
    "Western Samoa": (1997, "Samoa", "OFC"),
}

# =============================================================================
# ÉQUIPES MANQUANTES DU CLASSEMENT FIFA 2020
# Ces équipes existent mais ne sont pas dans le fichier FIFA téléchargé
# (membres FIFA suspendus, équipes non classées, etc.)
# =============================================================================

ADDITIONAL_TEAMS = {
    # Océanie (OFC) - membres non classés en 2020
    "Cook Islands": "OFC",
    "Tuvalu": "OFC", #ne peut pas participer à la coupe du monde mais membre associé de l'OFC
    "Tonga": "OFC",
    
    # Autres équipes potentiellement manquantes
    "Turks and Caicos": "CONCACAF",  # Turks and Caicos Islands
}

# Extraire les confédérations des équipes historiques
CONFEDERATIONS_HISTORICAL = {team: data[2] for team, data in HISTORICAL_TEAMS.items()}

# Fusionner : FIFA actuel + équipes historiques + équipes additionnelles
CONFEDERATIONS = {**CONFEDERATIONS_FIFA, **CONFEDERATIONS_HISTORICAL, **ADDITIONAL_TEAMS}

print("=" * 60)
print("ÉQUIPES HISTORIQUES ET ADDITIONNELLES AJOUTÉES")
print("=" * 60)
print(f"Total équipes avec confédération :")
print(f"  - FIFA actuel     : {len(CONFEDERATIONS_FIFA)}")
print(f"  - Historiques     : {len(CONFEDERATIONS_HISTORICAL)}")
print(f"  - Additionnelles  : {len(ADDITIONAL_TEAMS)}")
print(f"  - Total combiné   : {len(CONFEDERATIONS)}")

print("\n" + "-" * 60)
print("Liste des équipes historiques :")
print("-" * 60)
for team, (year, successor, conf) in HISTORICAL_TEAMS.items():
    successor_str = f"→ {successor}" if successor else "(pas de successeur)"
    print(f"  {team} ({conf}) - dissous en {year} {successor_str}")

print("\n" + "-" * 60)
print("Équipes additionnelles (non classées FIFA 2020) :")
print("-" * 60)
for team, conf in ADDITIONAL_TEAMS.items():
    print(f"  {team} ({conf})")

ÉQUIPES HISTORIQUES ET ADDITIONNELLES AJOUTÉES
Total équipes avec confédération :
  - FIFA actuel     : 210
  - Historiques     : 19
  - Additionnelles  : 4
  - Total combiné   : 232

------------------------------------------------------------
Liste des équipes historiques :
------------------------------------------------------------
  Soviet Union (UEFA) - dissous en 1991 → Russia
  Yugoslavia (UEFA) - dissous en 2003 → Serbia
  Czechoslovakia (UEFA) - dissous en 1993 → Czech Republic
  Serbia-Montenegro (UEFA) - dissous en 2006 → Serbia
  GDR (UEFA) - dissous en 1990 → Germany
  FRG (UEFA) - dissous en 1990 → Germany
  Saarland (UEFA) - dissous en 1956 → Germany
  Irish Free State (UEFA) - dissous en 1936 → Republic of Ireland
  Zaire (CAF) - dissous en 1997 → Congo DR
  Dahomey (CAF) - dissous en 1975 → Benin
  Upper Volta (CAF) - dissous en 1984 → Burkina Faso
  Rhodesia (CAF) - dissous en 1980 → Zimbabwe
  Dutch East Indies (AFC) - dissous en 1945 → Indonesia
  North Yemen (AFC)

## 6. Mapping des aliases vers noms standards

Chaque variante de nom (encodage, noms locaux, abréviations) est mappée explicitement vers le nom standard utilisé dans notre référentiel.

In [10]:
# =============================================================================
# MAPPING DES ALIASES VERS NOMS STANDARDS (NOMS FIFA)
# Chaque variante de nom est mappée explicitement vers le nom FIFA officiel
# Format : "nom_brut" -> "nom_standard_FIFA"
# =============================================================================

ALIASES_MAPPING = {
    # -------------------------------------------------------------------------
    # Corrections d'encodage (caractères corrompus)
    # -------------------------------------------------------------------------
    "C�te d'Ivoire": "Côte d'Ivoire",
    "Cï¿½te d'Ivoire": "Côte d'Ivoire",
    'rn">Bosnia and Herzegovina': "Bosnia and Herzegovina",
    '"rn"">Bosnia and Herzegovina"': "Bosnia and Herzegovina",

    # -------------------------------------------------------------------------
    # Variantes avec noms locaux (extraites du dataset 1930-2010)
    # Format original : "NomAnglais (NomLocal)" -> "Nom FIFA"
    # -------------------------------------------------------------------------
    "Afghanistan (افغانستان)": "Afghanistan",
    "Albania (Shqipëri)": "Albania",
    "Algeria (الجزائر)": "Algeria",
    "Armenia (Հайастан)": "Armenia",  # Forme cyrillique
    "Armenia (Հایاстан)": "Armenia",  # Forme correcte arménienne
    "Austria (Österreich)": "Austria",
    "Azerbaijan (Azərbaycan)": "Azerbaijan",
    "Bahrain (البحرين)": "Bahrain",
    "Bangladesh (বাংলাদেশ)": "Bangladesh",
    "Belarus (Беларусь)": "Belarus",
    "Belgium (België)": "Belgium",
    "Benin (Bénin)": "Benin",
    "Bosnia-Herzegovina (Bosna i Hercegovina)": "Bosnia and Herzegovina",
    "Brazil (Brasil)": "Brazil",
    "Brunei (بروني)": "Brunei Darussalam",  # Nom FIFA officiel
    "Bulgaria (България)": "Bulgaria",
    "Cambodia (កម្ពុជា)": "Cambodia",
    "Cameroon (Cameroun)": "Cameroon",
    "Cape Verde (Cabo Verde)": "Cabo Verde",  # Nom FIFA officiel
    "Central African Republic (Centrafrique)": "Central African Republic",
    "Chad (Tchad / تشاد)": "Chad",
    "China (中国)": "China PR",
    "Comoros (جزر القمر)": "Comoros",
    "Croatia (Hrvatska)": "Croatia",
    "Cyprus (Κύπρος)": "Cyprus",
    "Czech Republic (Česká Republika)": "Czech Republic",
    "Czechoslovakia (Československo)": "Czechoslovakia",
    "D.R. Congo (R.D. Congo)": "Congo DR",  # Nom FIFA officiel
    "Denmark (Danmark)": "Denmark",
    "Djibouti (جيبوتي)": "Djibouti",
    "Dominican Republic (República Dominicana)": "Dominican Republic",
    "Dutch Antilles (Nederlandse Antillen)": "Dutch Antilles",
    "Dutch East Indies (Nederlands-Indië)": "Dutch East Indies",
    "Dutch Guyana (Nederlands Guyana)": "Dutch Guyana",
    "East Timor (Timor-Leste)": "Timor-Leste",
    "Egypt (مصر)": "Egypt",
    "Equatorial Guinea (Guinea Ecuatorial)": "Equatorial Guinea",
    "Eritrea (ኤርትራ / إرتريا)": "Eritrea",
    "Estonia (Eesti)": "Estonia",
    # Ethiopia - version exacte depuis les données brutes
    "Ethiopia (ኢትዮⵒያ)": "Ethiopia",  # Version corrompue
    "Ethiopia (ኢትዮⵒ)": "Ethiopia",  # Version corrompue
    "Ethiopia (ኢትዮጵያ)": "Ethiopia",  # Forme correcte Ge'ez
    "FRG (BRD / Westdeutschland)": "FRG",
    "Faroe Islands (Føroyar)": "Faroe Islands",
    "Finland (Suomi)": "Finland",
    "GDR (DDR / Ostdeutschland)": "GDR",
    "Georgia (საქართველო)": "Georgia",
    "Germany (Deutschland)": "Germany",
    "Greece (Ελλάδα)": "Greece",
    "Guinea (Guinée)": "Guinea",
    "Guinea-Bissau (Guiné-Bissau)": "Guinea-Bissau",
    "Haiti (Haïti)": "Haiti",
    "Hong Kong (香港)": "Hong Kong",
    "Hungary (Magyarország)": "Hungary",
    "Iceland (Ísland)": "Iceland",
    "India (भारत)": "India",
    "Iran (ایران)": "IR Iran",  # Nom FIFA officiel
    "Iraq (العراق)": "Iraq",
    "Ireland (Éire)": "Republic of Ireland",  # Nom FIFA officiel
    "Irish Free State (Saorstát Éireann)": "Irish Free State",
    "Israel (ישראל)": "Israel",
    "Italy (Italia)": "Italy",
    # Ivory Coast - toutes les variantes d'apostrophe
    "Ivory Coast (Côte d'Ivoire)": "Côte d'Ivoire",  # Apostrophe droite
    "Ivory Coast (Côte d'Ivoire)": "Côte d'Ivoire",  # Apostrophe courbe
    "Japan (日本)": "Japan",
    "Jordan (الأردن)": "Jordan",
    "Kazakhstan (Қазақстан)": "Kazakhstan",
    "Kuwait (الكويت)": "Kuwait",
    "Kyrgyzstan (Кыргызстан)": "Kyrgyz Republic",  # Nom FIFA officiel
    "Laos (ນລາວ)": "Laos",
    "Latvia (Latvija)": "Latvia",
    "Lebanon (لبنان)": "Lebanon",
    "Libya (ليبيا)": "Libya",
    "Lithuania (Lietuva)": "Lithuania",
    "Luxembourg (Lëtzebuerg)": "Luxembourg",
    "Macao (澳门)": "Macau",
    "Macedonia (Македонија)": "North Macedonia",
    "Madagascar (Madagasikara)": "Madagascar",
    "Malawi (Malaŵi)": "Malawi",
    "Malaysia (مليسيا)": "Malaysia",
    "Maldives (Divehi Rājjēge)": "Maldives",
    "Mauritania (موريتانيا)": "Mauritania",
    "Mexico (México)": "Mexico",
    "Mongolia (Монгол Улс)": "Mongolia",
    "Montenegro (Црна Гора)": "Montenegro",
    "Morocco (المغرب)": "Morocco",
    "Mozambique (Moçambique)": "Mozambique",
    "Myanmar (ြမန်မာ)": "Myanmar",
    "Nepal (नेपाल)": "Nepal",
    "Netherlands (Nederland)": "Netherlands",
    "New Caledonia (Nouvelle-Calédonie)": "New Caledonia",
    "New Zealand (Aotearoa)": "New Zealand",
    "North Korea (조선)": "Korea DPR",
    "North Yemen (اليمن)": "North Yemen",
    "Northern Ireland (Ulster)": "Northern Ireland",
    "Norway (Norge)": "Norway",
    "Oman (عمان)": "Oman",
    "Pakistan (پاکستان)": "Pakistan",
    "Palestine (فلسطين)": "Palestine",
    "Panama (Panamá)": "Panama",
    "Papua New Guinea (Papua Niugini)": "Papua New Guinea",
    "Peru (Perú)": "Peru",
    "Philippines (Pilipinas)": "Philippines",
    "Poland (Polska)": "Poland",
    "Qatar (قطر)": "Qatar",
    "Romania (România)": "Romania",
    "Russia (Россия)": "Russia",
    "Saudi Arabia (العربية السعودية)": "Saudi Arabia",
    "Senegal (Sénégal)": "Senegal",
    "Serbia (Србија)": "Serbia",
    "Serbia-Montenegro (Србија и Црна Гора)": "Serbia-Montenegro",
    "Singapore (新加坡)": "Singapore",
    "Slovakia (Slovensko)": "Slovakia",
    "Slovenia (Slovenija)": "Slovenia",
    "Somalia (Soomaaliya)": "Somalia",
    "South Africa (Suid-Afrika)": "South Africa",
    "South Korea (한국)": "Korea Republic",
    "South Vietnam (Việt Nam)": "South Vietnam",
    "South Yemen (اليمن)": "South Yemen",
    "Soviet Union (СССР)": "Soviet Union",
    "Spain (España)": "Spain",
    "Sri Lanka (ශ්රී ලංකාව)": "Sri Lanka",
    "Sudan (السودان)": "Sudan",
    "Surinam (Suriname)": "Suriname",
    "Swaziland (Swatini)": "Swaziland",  # Nom FIFA (pas Eswatini)
    "Sweden (Sverige)": "Sweden",
    "Switzerland (Schweiz / Suisse)": "Switzerland",
    "Syria (سوريا)": "Syria",
    "Taiwan (台湾)": "Chinese Taipei",
    "Tajikistan (Точикистон)": "Tajikistan",
    "Thailand (ประเทศไทย)": "Thailand",
    "Tunisia (تونس)": "Tunisia",
    "Turkey (Türkiye)": "Turkey",  # Nom FIFA actuel
    "Turkmenistan (Türkmenistan)": "Turkmenistan",
    "Ukraine (Україна)": "Ukraine",
    "United Arab Emirates (الإمارات العربية المتحدة)": "United Arab Emirates",
    "Upper Volta (Haute-Volta)": "Upper Volta",
    "Uzbekistan (Ўзбекистон)": "Uzbekistan",
    "Vietnam (Việt Nam)": "Vietnam",
    "Wales (Cymru)": "Wales",
    "Yemen (اليمن)": "Yemen",
    "Yugoslavia (Југославија)": "Yugoslavia",
    "Zaire (Zaïre)": "Zaire",

    # -------------------------------------------------------------------------
    # Aliases spéciaux (noms alternatifs utilisés dans certains datasets)
    # -------------------------------------------------------------------------
    "IR Iran": "IR Iran",  # Nom FIFA officiel (garder tel quel)
    "Iran": "IR Iran",  # Mapper vers le nom FIFA
    "Korea Republic": "Korea Republic",
    "South Korea": "Korea Republic",
    
    # -------------------------------------------------------------------------
    # Aliases manquants identifiés lors de la validation
    # -------------------------------------------------------------------------
    "Antigua": "Antigua and Barbuda",
    "Guayana": "Guyana",  # Faute de frappe probable
    "Saint Kitts & Nevis": "St. Kitts and Nevis",  # Nom FIFA
    "Saint Lucia": "St. Lucia",  # Nom FIFA
    "Saint Vincent & The Grenadines": "St. Vincent / Grenadines",  # Nom FIFA
    "São Tomé e Príncipe": "São Tomé and Príncipe",  # Nom FIFA
    "United States": "USA",
    "Cote d'Ivoire": "Côte d'Ivoire",  # Sans accent
    "Serbia and Montenegro": "Serbia-Montenegro",
    "KOREA REPUBLIC": "Korea Republic",  # Version majuscules
}

print(f"Nombre d'aliases définis : {len(ALIASES_MAPPING)}")

Nombre d'aliases définis : 160


In [11]:
# =============================================================================
# PLACEHOLDERS À EXCLURE (pas des équipes réelles)
# Ces valeurs apparaissent dans les données brutes pour les matchs à venir
# ou les phases de poules non encore jouées
# =============================================================================

PLACEHOLDERS = {
    # Numéros de positions
    "1", "2", "3", "4", "5", "6", "7", "8",
    # Lettres de groupes
    "A", "B", "C", "D",
    # Identifiants de poules
    "A1", "A2", "B1", "B2", "C1", "C2", "D1", "D2",
    "E1", "E2", "F1", "F2", "G1", "G2", "H1", "H2",
    # Marqueurs de progression
    "WINNER X", "WINNER Y", "LOSER X", "LOSER Y",
}

print(f"Nombre de placeholders définis : {len(PLACEHOLDERS)}")
print(f"\nPlaceholders : {sorted(PLACEHOLDERS)}")

Nombre de placeholders définis : 32

Placeholders : ['1', '2', '3', '4', '5', '6', '7', '8', 'A', 'A1', 'A2', 'B', 'B1', 'B2', 'C', 'C1', 'C2', 'D', 'D1', 'D2', 'E1', 'E2', 'F1', 'F2', 'G1', 'G2', 'H1', 'H2', 'LOSER X', 'LOSER Y', 'WINNER X', 'WINNER Y']


## 7. Fonctions de normalisation

In [12]:
# =============================================================================
# FONCTIONS DE NORMALISATION
# =============================================================================

import re

def normalize_team_name(raw_name: str) -> str | None:
    """
    Convertit un nom brut en nom standard.
    Retourne None si c'est un placeholder.
    
    Args:
        raw_name: Nom brut de l'équipe (peut contenir des caractères spéciaux)
    
    Returns:
        Nom standard ou None si placeholder
    """
    if pd.isna(raw_name):
        return None

    name = str(raw_name).strip()

    # Exclure les placeholders
    if name in PLACEHOLDERS:
        return None

    # Appliquer le mapping des aliases (correspondance exacte)
    if name in ALIASES_MAPPING:
        return ALIASES_MAPPING[name]
    
    # -------------------------------------------------------------------------
    # GESTION DYNAMIQUE DES CAS PROBLÉMATIQUES
    # Pour les équipes avec caractères spéciaux difficiles à mapper exactement
    # -------------------------------------------------------------------------
    
    # Cas 1: Ethiopia avec différentes formes de l'écriture Ge'ez
    if name.startswith("Ethiopia ("):
        return "Ethiopia"
    
    # Cas 2: Ivory Coast avec différentes apostrophes (droite ' ou courbe ')
    if name.startswith("Ivory Coast (Côte d"):
        return "Côte d'Ivoire"
    
    # Cas 3: Armenia avec différentes formes arméniennes
    if name.startswith("Armenia ("):
        return "Armenia"

    return name


def get_confederation(team_name: str) -> str | None:
    """
    Retourne la confédération d'une équipe.
    
    Args:
        team_name: Nom de l'équipe (brut ou normalisé)
    
    Returns:
        Code de confédération (UEFA, CONMEBOL, CAF, AFC, CONCACAF, OFC) ou None
    """
    normalized = normalize_team_name(team_name)
    if normalized is None:
        return None

    # Chercher dans le dictionnaire des confédérations
    return CONFEDERATIONS.get(normalized, None)


def get_aliases(team_name: str) -> list[str]:
    """
    Retourne tous les aliases connus pour une équipe.
    
    Args:
        team_name: Nom standard de l'équipe
    
    Returns:
        Liste des aliases (noms alternatifs)
    """
    aliases = []
    for alias, standard in ALIASES_MAPPING.items():
        if standard == team_name:
            aliases.append(alias)
    return aliases


def is_historical_team(team_name: str) -> bool:
    """
    Vérifie si l'équipe est une équipe historique dissoute.
    
    Args:
        team_name: Nom de l'équipe
    
    Returns:
        True si l'équipe est historique (dissoute)
    """
    return team_name in HISTORICAL_TEAMS


def get_successor(team_name: str) -> str | None:
    """
    Retourne le successeur FIFA d'une équipe dissoute.
    
    Args:
        team_name: Nom de l'équipe historique
    
    Returns:
        Nom du successeur FIFA ou None
    """
    if team_name in HISTORICAL_TEAMS:
        return HISTORICAL_TEAMS[team_name][1]
    return None


def get_dissolution_year(team_name: str) -> int | None:
    """
    Retourne l'année de dissolution d'une équipe historique.
    
    Args:
        team_name: Nom de l'équipe historique
    
    Returns:
        Année de dissolution ou None
    """
    if team_name in HISTORICAL_TEAMS:
        return HISTORICAL_TEAMS[team_name][0]
    return None


# Test des fonctions
print("=" * 60)
print("TESTS DES FONCTIONS DE NORMALISATION")
print("=" * 60)

test_cases = [
    "Brazil (Brasil)",
    "Soviet Union (СССР)",
    "A1",
    "Germany",
    "IR Iran",
    "C�te d'Ivoire",
    "Ethiopia (ኢትዮⵒያ)",  # Test du cas Ethiopia
    "Ivory Coast (Côte d'Ivoire)",  # Test du cas Ivory Coast
]

for test in test_cases:
    normalized = normalize_team_name(test)
    conf = get_confederation(test)
    is_hist = is_historical_team(normalized) if normalized else False
    print(f"'{test}' -> '{normalized}' | Conf: {conf} | Historique: {is_hist}")

TESTS DES FONCTIONS DE NORMALISATION
'Brazil (Brasil)' -> 'Brazil' | Conf: CONMEBOL | Historique: False
'Soviet Union (СССР)' -> 'Soviet Union' | Conf: UEFA | Historique: True
'A1' -> 'None' | Conf: None | Historique: False
'Germany' -> 'Germany' | Conf: UEFA | Historique: False
'IR Iran' -> 'IR Iran' | Conf: AFC | Historique: False
'C�te d'Ivoire' -> 'Côte d'Ivoire' | Conf: CAF | Historique: False
'Ethiopia (ኢትዮⵒያ)' -> 'Ethiopia' | Conf: CAF | Historique: False
'Ivory Coast (Côte d'Ivoire)' -> 'Côte d'Ivoire' | Conf: CAF | Historique: False


## 8. Construction du référentiel des équipes

On construit un dictionnaire complet pour chaque équipe avec :
- Sa confédération
- Ses aliases (variantes de noms)
- Son statut historique (si équipe dissoute)
- Son successeur FIFA (le cas échéant)

In [13]:
# =============================================================================
# CONSTRUCTION DU RÉFÉRENTIEL TEAMS
# Génération du dictionnaire complet à partir des données définies ci-dessus
# =============================================================================

def extract_variants(alias: str) -> list[str]:
    """
    Extrait les variantes d'un alias.
    Exemple: "China (中国)" -> ["China (中国)", "China"]
    
    Args:
        alias: L'alias original (ex: "Brazil (Brasil)")
    
    Returns:
        Liste des variantes incluant l'alias original et la partie avant parenthèse
    """
    variants = [alias]
    if "(" in alias:
        # Extraire la partie avant la parenthèse
        base_name = alias.split("(")[0].strip()
        if base_name and base_name != alias:
            variants.append(base_name)
    return variants


def build_teams_reference() -> dict:
    """
    Construit le référentiel complet des équipes.
    
    Returns:
        Dictionnaire {nom_équipe: {confederation, aliases, is_historical, ...}}
    """
    teams_ref = {}

    # 1. Ajouter toutes les équipes du classement FIFA actuel
    for team_name, confederation in CONFEDERATIONS.items():
        if team_name not in teams_ref:
            teams_ref[team_name] = {
                "confederation": confederation,
                "aliases": get_aliases(team_name),
                "is_historical": is_historical_team(team_name),
                "fifa_successor": None,
                "dissolved_year": None,
            }

    # 2. Compléter les informations des équipes historiques
    for team_name, (dissolved_year, successor, conf) in HISTORICAL_TEAMS.items():
        if team_name in teams_ref:
            teams_ref[team_name]["dissolved_year"] = dissolved_year
            teams_ref[team_name]["fifa_successor"] = successor
        else:
            # Équipe historique pas encore dans le ref
            teams_ref[team_name] = {
                "confederation": conf,
                "aliases": get_aliases(team_name),
                "is_historical": True,
                "fifa_successor": successor,
                "dissolved_year": dissolved_year,
            }

    # 3. Enrichir les aliases avec les variantes extraites
    # Ex: "China (中国)" génère aussi "China" comme alias
    for team_name in teams_ref:
        original_aliases = teams_ref[team_name]['aliases']
        enriched_aliases = []
        for alias in original_aliases:
            enriched_aliases.extend(extract_variants(alias))
        # Dédupliquer et garder l'ordre
        teams_ref[team_name]['aliases'] = list(dict.fromkeys(enriched_aliases))

    return teams_ref


# Construire le référentiel
teams_reference = build_teams_reference()

# Afficher les statistiques
print("=" * 60)
print("STATISTIQUES DU RÉFÉRENTIEL")
print("=" * 60)
print(f"Total équipes dans le référentiel : {len(teams_reference)}")
print(f"Équipes historiques : {sum(1 for t in teams_reference.values() if t['is_historical'])}")
print(f"Équipes avec aliases : {sum(1 for t in teams_reference.values() if t['aliases'])}")

# Compter le nombre total d'aliases (avec variantes)
total_aliases = sum(len(t['aliases']) for t in teams_reference.values())
print(f"Total aliases (avec variantes extraites) : {total_aliases}")

# Répartition par confédération
print("\n" + "-" * 60)
print("RÉPARTITION PAR CONFÉDÉRATION")
print("-" * 60)
conf_counts = {}
for team_data in teams_reference.values():
    conf = team_data['confederation']
    conf_counts[conf] = conf_counts.get(conf, 0) + 1

for conf, count in sorted(conf_counts.items(), key=lambda x: -x[1]):
    print(f"  {conf}: {count} équipes")

# Exemples de variantes extraites
print("\n" + "-" * 60)
print("EXEMPLES DE VARIANTES EXTRAITES")
print("-" * 60)
examples_with_variants = ["China PR", "IR Iran", "Korea Republic", "Côte d'Ivoire"]
for team in examples_with_variants:
    if team in teams_reference:
        aliases = teams_reference[team]['aliases']
        print(f"  {team}: {aliases}")

STATISTIQUES DU RÉFÉRENTIEL
Total équipes dans le référentiel : 232
Équipes historiques : 19
Équipes avec aliases : 146
Total aliases (avec variantes extraites) : 297

------------------------------------------------------------
RÉPARTITION PAR CONFÉDÉRATION
------------------------------------------------------------
  UEFA: 63 équipes
  CAF: 58 équipes
  AFC: 50 équipes
  CONCACAF: 37 équipes
  OFC: 13 équipes
  CONMEBOL: 11 équipes

------------------------------------------------------------
EXEMPLES DE VARIANTES EXTRAITES
------------------------------------------------------------
  China PR: ['China (中国)', 'China']
  IR Iran: ['Iran (ایران)', 'Iran', 'IR Iran']
  Korea Republic: ['South Korea (한국)', 'South Korea', 'Korea Republic', 'KOREA REPUBLIC']
  Côte d'Ivoire: ["C�te d'Ivoire", "Cï¿½te d'Ivoire", "Ivory Coast (Côte d'Ivoire)", 'Ivory Coast', "Cote d'Ivoire"]


In [14]:
# Aperçu de quelques entrées du référentiel
print("=" * 60)
print("EXEMPLES D'ENTRÉES DU RÉFÉRENTIEL")
print("=" * 60)

# Montrer des exemples variés
examples = ["France", "Brazil", "Soviet Union", "Germany", "Ivory Coast"]
for team in examples:
    if team in teams_reference:
        data = teams_reference[team]
        print(f"\n{team}:")
        print(f"  Confédération : {data['confederation']}")
        print(f"  Aliases       : {data['aliases'][:3]}..." if len(data['aliases']) > 3 else f"  Aliases       : {data['aliases']}")
        print(f"  Historique    : {data['is_historical']}")
        if data['is_historical']:
            print(f"  Dissous       : {data['dissolved_year']}")
            print(f"  Successeur    : {data['fifa_successor']}")

EXEMPLES D'ENTRÉES DU RÉFÉRENTIEL

France:
  Confédération : UEFA
  Aliases       : []
  Historique    : False

Brazil:
  Confédération : CONMEBOL
  Aliases       : ['Brazil (Brasil)', 'Brazil']
  Historique    : False

Soviet Union:
  Confédération : UEFA
  Aliases       : ['Soviet Union (СССР)', 'Soviet Union']
  Historique    : True
  Dissous       : 1991
  Successeur    : Russia

Germany:
  Confédération : UEFA
  Aliases       : ['Germany (Deutschland)', 'Germany']
  Historique    : False


## 9. Export du référentiel en JSON

Le fichier JSON généré servira de source unique de vérité pour le mapping des équipes dans tout le projet.

In [15]:
# =============================================================================
# EXPORT DU RÉFÉRENTIEL EN JSON
# Ce fichier est généré automatiquement depuis le notebook
# Pour modifier le mapping, modifier les dictionnaires ci-dessus et ré-exécuter
# =============================================================================

from pathlib import Path

# Créer le dossier reference s'il n'existe pas
reference_path = Path('../data/reference')
reference_path.mkdir(parents=True, exist_ok=True)

# Exporter le référentiel
output_file = reference_path / 'teams_mapping.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(teams_reference, f, ensure_ascii=False, indent=2)

print("=" * 60)
print("EXPORT TERMINÉ")
print("=" * 60)
print(f"Fichier exporté : {output_file.resolve()}")
print(f"Taille du fichier : {output_file.stat().st_size / 1024:.1f} KB")
print(f"Nombre d'équipes : {len(teams_reference)}")

EXPORT TERMINÉ
Fichier exporté : C:\Users\moi\Desktop\projetsimplon\Brief-2-ETL-donnees-footballistiques-Short-Kings\data\reference\teams_mapping.json
Taille du fichier : 45.1 KB
Nombre d'équipes : 232


## 10. Validation du mapping

Vérification que toutes les équipes présentes dans nos datasets ont un mapping valide vers le référentiel.

In [16]:
# =============================================================================
# VALIDATION : Toutes les équipes des matchs ont un mapping
# =============================================================================

print("=" * 60)
print("VALIDATION DU MAPPING")
print("=" * 60)

# Récupérer toutes les équipes brutes (recalculées pour être sûr)
all_raw_teams = equipes_1930_2010 | equipes_2014 | equipes_2018

# Vérifier le mapping de chaque équipe
missing_mapping = []
missing_confederation = []

for team in all_raw_teams:
    normalized = normalize_team_name(team)
    
    # Si c'est un placeholder, on ignore
    if normalized is None:
        continue
    
    # Vérifier si l'équipe normalisée existe dans le référentiel
    if normalized not in teams_reference:
        missing_mapping.append((team, normalized))
    elif teams_reference[normalized]['confederation'] is None:
        missing_confederation.append(normalized)

# Afficher les résultats
print(f"\nÉquipes brutes analysées : {len(all_raw_teams)}")
print(f"Placeholders exclus : {len([t for t in all_raw_teams if normalize_team_name(t) is None])}")

if missing_mapping:
    print(f"\n{len(missing_mapping)} équipes sans mapping dans le référentiel :")
    for raw, norm in sorted(missing_mapping)[:20]:
        print(f"  - '{raw}' -> '{norm}'")
    if len(missing_mapping) > 20:
        print(f"  ... et {len(missing_mapping) - 20} autres")
else:
    print("\nToutes les équipes ont un mapping vers le référentiel !")

if missing_confederation:
    print(f"\n{len(missing_confederation)} équipes sans confédération :")
    for team in sorted(missing_confederation)[:10]:
        print(f"  - '{team}'")
else:
    print("Toutes les équipes ont une confédération assignée !")

VALIDATION DU MAPPING

Équipes brutes analysées : 294
Placeholders exclus : 32

Toutes les équipes ont un mapping vers le référentiel !
Toutes les équipes ont une confédération assignée !


## 11. Normalisation du nom des équipes et insertion de la confédération

In [17]:
import sys
sys.path.append('../src')

from normalize_teams import normalize_teams

# Exécuter la normalisation
df_result, unmatched = normalize_teams()

Fichier créé : C:\Users\moi\Desktop\projetsimplon\Brief-2-ETL-donnees-footballistiques-Short-Kings\notebooks\..\data\processed\teams_traitees.csv
Nombre d'équipes : 229
Équipes normalisées : 229

Toutes les équipes ont été matchées !

Exemples de normalisation :
  Antigua → Antigua and Barbuda
  Bosnia-Herzegovina → Bosnia and Herzegovina
  Brunei → Brunei Darussalam
  Cape Verde → Cabo Verde
  China → China PR
  D.R. Congo → Congo DR
  East Timor → Timor-Leste
  Guayana → Guyana
  Iran → IR Iran
  Ireland → Republic of Ireland


## 12. Mise à jour de la base de données PostgreSQL

Mise à jour de la table `teams` avec les confédérations et aliases pour chaque équipe.

In [ ]:
# =============================================================================
# MISE À JOUR DE LA TABLE TEAMS EN BASE DE DONNÉES
# =============================================================================

def update_teams_in_db(engine, teams_ref: dict):
    """
    Met à jour la table teams avec les confédérations et aliases.
    
    Args:
        engine: Connexion SQLAlchemy
        teams_ref: Dictionnaire du référentiel des équipes
    """
    updated_count = 0
    not_found = []
    
    with engine.begin() as conn:
        for team_name, data in teams_ref.items():
            # Convertir aliases en JSON string pour PostgreSQL
            aliases_json = json.dumps(data['aliases'], ensure_ascii=False)
            
            # Essayer de mettre à jour la ligne existante
            result = conn.execute(text("""
                UPDATE teams
                SET confederation = :conf,
                    aliases = :aliases::jsonb
                WHERE nom_standard = :name
            """), {
                'conf': data['confederation'],
                'aliases': aliases_json,
                'name': team_name
            })
            
            if result.rowcount > 0:
                updated_count += 1
            else:
                not_found.append(team_name)
    
    return updated_count, not_found


print("=" * 60)
print("MISE À JOUR DE LA BASE DE DONNÉES")
print("=" * 60)

# Exécuter la mise à jour
updated, not_found = update_teams_in_db(engine, teams_reference)

print(f"\n{updated} équipes mises à jour dans la BDD")

if not_found:
    print(f"\n{len(not_found)} équipes du référentiel non trouvées en BDD :")
    for team in sorted(not_found)[:10]:
        print(f"  - '{team}'")
    if len(not_found) > 10:
        print(f"  ... et {len(not_found) - 10} autres")

In [ ]:
# =============================================================================
# VÉRIFICATION DE LA MISE À JOUR EN BDD
# =============================================================================

print("=" * 60)
print("VÉRIFICATION EN BASE DE DONNÉES")
print("=" * 60)

with engine.connect() as conn:
    # Répartition par confédération
    result = conn.execute(text("""
        SELECT confederation, COUNT(*) as count
        FROM teams
        WHERE confederation IS NOT NULL
        GROUP BY confederation
        ORDER BY count DESC
    """))
    
    print("\nRépartition par confédération en BDD :")
    for row in result:
        print(f"  {row[0]}: {row[1]} équipes")
    
    # Nombre d'équipes avec aliases
    result = conn.execute(text("""
        SELECT COUNT(*) 
        FROM teams 
        WHERE aliases IS NOT NULL AND aliases != '[]'::jsonb
    """))
    count_with_aliases = result.scalar()
    print(f"\nÉquipes avec aliases : {count_with_aliases}")
    
    # Exemple d'équipes
    print("\n" + "-" * 60)
    print("EXEMPLES D'ÉQUIPES EN BDD")
    print("-" * 60)
    result = conn.execute(text("""
        SELECT nom_standard, confederation, aliases
        FROM teams
        WHERE confederation IS NOT NULL
        LIMIT 5
    """))
    for row in result:
        print(f"  {row[0]} | {row[1]} | {row[2]}")